
# Image Retraining -2 还是用tensorflow

主要是做练习时候的一个困惑： 

使用已经训练好的 InceptionV3，对新的图片分类问题进行训练，**很容易** 产生过拟合：对训练集，很容易达到100%的准确率，但是对测试集/验证集，准确率不到 50%，并且很难继续提升。 这说明什么，说明确实学习到了图片的特征，但是学习能力太强，产生了过拟合。

google了一下，有些参考：

- https://stackoverflow.com/questions/37605611/would-adding-dropout-help-reduce-overfitting-when-following-tensorflows-transfe

- [Image Retraining](https://www.tensorflow.org/tutorials/image_retraining)

- [tensorboard](https://www.youtube.com/watch?time_continue=1426&v=eBbEDRsCmv4) ，这个视频说了tensorboard的一些特性。例子里面用了不少 summary，用于观察

这里按照 Image Retraining的教程，推演一遍，看看如何解决这个问题

main函数步骤：

1. 准备一些模型参数
2. 准备图片文件，包括训练集、验证集、测试集，以及标签
3. 准备模型
    1. 加载inception_v3模型
    2. 在原模型的某一层（bottleneck）后面添加 新的模型层（ retrain ），用于新的分类任务
    3. 在原模型的前面加上 `jepg_decoding`层，用于jpeg图片的解码。或者加上 `distortion`层，用于data argumentation。
    4. 如果不是用`distortion`，则计算并缓存所有图片在原模型上bottleneck层的输出
    5. 在新的训练层之后添加 评估计算（`add_evaluation_step`）。
4. 训练模型：使用 bottleneck层输出的缓存或者直接计算，以及其他信息，开始训练。
5. 其他操作：存储模型


一些Tips：

1. 需要缓存一些bottleneck输出，最好缓存在服务器上，而不是本地。 这样训练的时候取batch速度会快一些。训练会变快，很有效！
2. 还是不理解tensorflow的session和graph。 在session运行完之后，不能清除GPU内存，非常不方便！
3. 原文件的做法是 将训练后的模型彻底保存下来，然后读取，然后在测试集上评估。
4. 对于tensorflow的placeholder，有没有什么不赋值就用默认值的用法？ 有 `placeholder_with_default`
5. 使用dropout提升的效果并不明显。 一些观察：准确率评估不是很稳定。 后续可以尝试一下 data argumentation。


In [1]:
import os
import re
import hashlib
import sys
import urllib
import tarfile
import random
import scipy.io

import numpy as np
import tensorflow as tf
import keras.backend as K

from datetime import datetime
from tensorflow.python.util import compat
from tensorflow.python.platform import gfile

def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
    
limit_mem()


/home/yangdong/Applications/miniconda3/envs/DL3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 1. 模型参数

In [2]:
# model_info
model_info =  {
    'data_url': 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz',
    'resized_input_tensor_name': 'Mul:0',
    'bottleneck_tensor_name': 'pool_3/_reshape:0',
    'bottleneck_tensor_size': 2048,
    'class_count': 5,   # class of new task
    'input_width': 299,
    'input_height': 299,
    'input_depth': 3,
    'model_file_name': 'classify_image_graph_def.pb',
    'input_mean': 128,
    'input_std': 128,
    'quantize_layer': False,
}

# configs
class FLAGS:
    model_dir = '/home/yangdong/Downloads/tmp/imagenet/'
    bottleneck_dir = '/tmp/bottleneck/'
    summaries_dir  = '/tmp/retrain_logs'
    final_tensor_name = 'final_result'
    learning_rate = 0.01
    how_many_training_steps = 5000
    eval_step_interval = 100
    train_batch_size = 100
    validation_batch_size = 100
    test_batch_size = -1
    
    l2_penalty = 0.01

CHECKPOINT_NAME = '/tmp/_retrain_checkpoint'

## 2. 准备训练数据

这里是102flowers数据，包括102种花。

来源： <http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html>

准备训练集、验证集和测试集以及标签。

输出形式：  文件名， 标签（数字）， 标签名
分成 训练集、验证集和测试集，或者不要验证集。

下一步需要输出： bottlenecks 文件。

直接映射。训练集、验证集只返回索引。

In [3]:
MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M

image_dir = './flower_photos'
flowers102_tar_path = "../readonly/week3/102flowers.tgz"
labels_mat_path = '../readonly/week3/imagelabels.mat'
tmp_flowers_path = '/tmp/flowers/'
tmp_bottleneck_path = '/tmp/flowers/bottlenecks'

def ensure_dir_exists(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
ensure_dir_exists(tmp_flowers_path)
ensure_dir_exists(tmp_bottleneck_path)

if not os.path.exists(os.path.join(tmp_flowers_path, 'jpg')):
    tarfile.open(flowers102_tar_path, 'r:gz').extractall(tmp_flowers_path)

with tarfile.open(flowers102_tar_path) as f:
    all_files = sorted([m.name for m in f.getmembers() if m.isfile()])

all_labels = scipy.io.loadmat(labels_mat_path)['labels'][0] - 1

def get_image_path(image_file):
    return os.path.join(tmp_flowers_path, image_file)


NUM_CLASSES = len(np.unique(all_labels))
print(NUM_CLASSES)

102


In [4]:
from sklearn.model_selection import train_test_split
# split into train/test
tr_files, te_files, tr_labels, te_labels = \
    train_test_split(all_files, all_labels, test_size=0.2, random_state=42, stratify=all_labels)

## 3. 准备模型

### 3.1 使用 inception_v3模型

tensorflow的model文件，参考这个文章： <https://www.tensorflow.org/extend/tool_developers/>

In [5]:

# 下载 inception_v3 模型文件 (.pb文件)

data_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
model_dir = './tmp/imagenet'
bottleneck_dir = '/tmp/bottleneck'
model_file_name = 'classify_image_graph_def.pb'
summaries_dir = '/tmp/retrain_logs/'

# 如果没有 './tmp/imagenet' 文件夹则创建
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
filename = data_url.split('/')[-1]
filepath = os.path.join(model_dir, filename)

# 进度条
def _progress(count, block_size, total_size):
    sys.stdout.write('\r>> Downloading %s %.1f%%' %
                     (filename,
                      float(count * block_size) / float(total_size) * 100.0))
    sys.stdout.flush()
    
# 没有文件则下载
if not os.path.exists(filepath):
    filepath, _ = urllib.request.urlretrieve(data_url, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    tf.logging.info('Successfully downloaded %s %d bytes.', filename, statinfo.st_size)

# 没有模型文件，则解压缩
model_path = os.path.join(model_dir, model_file_name)
if not os.path.exists(model_path):
    print('Extracting file from ', filepath)
    tarfile.open(filepath, 'r:gz').extractall(model_dir)
    

In [6]:

# 从pb文件里面，创建 graph：
print('Model path: ', model_path)

def create_model_graph():
    tf.reset_default_graph()
    with tf.Graph().as_default() as graph:
        print(graph)
        with gfile.FastGFile(model_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            bottleneck_tensor, resized_input_tensor = (tf.import_graph_def(
                graph_def,
                name='',
                return_elements=['pool_3/_reshape:0', 'Mul:0']
            ))
    return graph, bottleneck_tensor, resized_input_tensor

graph, bottleneck_tensor, resized_input_tensor =  create_model_graph()

Model path:  ./tmp/imagenet/classify_image_graph_def.pb


In [7]:
graph

In [8]:
print(resized_input_tensor)
print(bottleneck_tensor)

Tensor("Mul:0", shape=(1, 299, 299, 3), dtype=float32)
Tensor("pool_3/_reshape:0", shape=(1, 2048), dtype=float32)


### 3.2 添加新的训练层

In [9]:

bottleneck_tensor_size = int(bottleneck_tensor.shape[-1])
class_count = NUM_CLASSES

In [10]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

with graph.as_default():
    # BottleneckInputPlaceholder, input =  bottlenck output
    with tf.name_scope('input'):
        bottleneck_input = tf.placeholder_with_default(
            bottleneck_tensor,
            shape=[None, bottleneck_tensor_size],
            name='BottleneckInputPlaceholder')

        ground_truth_input = tf.placeholder(
            tf.int64, [None], name='GroundTruthInput')

    # Organizing the following ops so they are easier to see in TensorBoard.
    with tf.name_scope('final_retrain_ops'):
        with tf.name_scope('weights'):
            initial_value = tf.truncated_normal(
                [bottleneck_tensor_size, class_count], stddev=0.001)
            layer_weights = tf.Variable(initial_value, name='final_weights')
            variable_summaries(layer_weights)

        with tf.name_scope('biases'):
            layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
            variable_summaries(layer_biases)
            
        # add dropout
        with tf.name_scope('dropout'):
            keep_prob = tf.placeholder_with_default(tf.constant(1.0), [], name='keep_prob')
            drop = tf.nn.dropout(bottleneck_input, keep_prob)
            
        with tf.name_scope('Wx_plus_b'):
            logits = tf.matmul(drop, layer_weights) + layer_biases
            tf.summary.histogram('pre_activations', logits)

    # final output
    final_tensor = tf.nn.softmax(logits, name='final_result')
    tf.summary.histogram('activations', final_tensor)
    
# here we have : (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input, final_tensor)


In [11]:
# cross entropy 和 train：
with graph.as_default():
    # cross entropy
    with tf.name_scope('cross_entropy'):
        cross_entropy_mean = tf.losses.sparse_softmax_cross_entropy(
            labels=ground_truth_input, logits=logits)
    tf.summary.scalar('cross_entropy', cross_entropy_mean)

    with tf.name_scope('loss'):
        regularizer = tf.nn.l2_loss(layer_weights)
        loss = cross_entropy_mean + FLAGS.l2_penalty * regularizer
        
    # train step, optimizer
    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)
        train_step = optimizer.minimize(loss)

In [12]:
regularizer.shape

TensorShape([])

In [13]:
# 评估层
with graph.as_default():
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            prediction = tf.argmax(final_tensor, 1)
            correct_prediction = tf.equal(prediction, ground_truth_input)
        with tf.name_scope('accuracy'):
            evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', evaluation_step)


### 3.3 在原模型前面加上 `jepg_decoding` 层，用于处理图片输入



In [14]:
# 在输入层添加 jpeg decoding层

input_width = 299
input_height = 299
input_depth = 3
input_mean = 128   # 256/2
input_std = 128    # 256/2
# resize and normalize ~(-0.5, 0.5)

with graph.as_default():
    jpeg_data_tensor = tf.placeholder(tf.string, name='DecodeJPGInput')
    decoded_image = tf.image.decode_jpeg(jpeg_data_tensor, channels=input_depth)
    decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
    decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
    resize_shape = tf.stack([input_height, input_width])
    resize_shape_as_int = tf.cast(resize_shape, dtype=tf.int32)
    resized_image = tf.image.resize_bilinear(decoded_image_4d,
                                             resize_shape_as_int)
    offset_image = tf.subtract(resized_image, input_mean)
    decoded_image_tensor = tf.multiply(offset_image, 1.0 / input_std)
    


In [15]:
jpeg_data_tensor.graph

### 3.4 计算并缓存 bottleneck 的输出（对所有图片）


In [16]:
# 计算 bottleneck层 的输出，并缓存。 主要是为了避免重复计算，加快训练速度

    
def run_bottleneck_on_image(sess, image_data):
    # image_data_tensor -> decoded_image_tensor | resized_input_tensor -> bottleneck_tensor
    # image_data -> resized_input_values -> bottleneck_values
    resized_input_values = sess.run(decoded_image_tensor, {jpeg_data_tensor: image_data})
    # Then run it through the recognition network.
    bottleneck_values = sess.run(bottleneck_tensor, {resized_input_tensor: resized_input_values})
    bottleneck_values = np.squeeze(bottleneck_values)
    return bottleneck_values

def create_bottleneck_file(sess, image_path, bottleneck_path):
    if not gfile.Exists(image_path):
        tf.logging.fatal('File does not exist %s', image_path)
    image_data = gfile.FastGFile(image_path, 'rb').read()
    try:
        bottleneck_values = run_bottleneck_on_image(sess, image_data)
    except Exception as e:
        raise RuntimeError('Error during processing file %s (%s)' % (image_path, str(e)))
        
    bottleneck_string = ','.join(str(x) for x in bottleneck_values)
    with open(bottleneck_path, 'w') as bottleneck_file:
        bottleneck_file.write(bottleneck_string)


def get_or_create_bottleneck(sess, index):
    image_path = os.path.join(tmp_flowers_path, all_files[index])
    image_name = os.path.basename(all_files[index])
    bottleneck_path = os.path.join(tmp_bottleneck_path, image_name + '.txt')
    
    if not os.path.exists(bottleneck_path):
        create_bottleneck_file(sess, image_path, bottleneck_path)
    with open(bottleneck_path, 'r') as bottleneck_file:
        bottleneck_string = bottleneck_file.read()
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
    return bottleneck_values
    
    
def cache_bottlenecks(sess):
    how_many_bottlenecks = 0
    for i in range(len(all_files)):
        get_or_create_bottleneck(sess, i)
        how_many_bottlenecks += 1
        if how_many_bottlenecks % 100 == 0:
            tf.logging.info(str(how_many_bottlenecks) + ' bottleneck files created.')
        
#         for category in ['training', 'testing', 'validation']:
#             category_list = label_lists[category]
#             for index, unused_base_name in enumerate(category_list):
#                 get_or_create_bottleneck(sess, label_name, unused_base_name)
#                 how_many_bottlenecks += 1
# #                 if how_many_bottlenecks % 100 == 0:
# #                     tf.logging.info(str(how_many_bottlenecks) + ' bottleneck files created.')
                    

In [17]:
# with tf.Session(graph=graph) as sess:
#     # cache bottlenecks
#     cache_bottlenecks(sess)

In [18]:
os.path.basename(all_files[-1])

'image_08189.jpg'



## 4. 开始训练



In [19]:

# 拆分数据集

indexes = np.random.permutation(len(all_files))
split_idx = -int(len(all_files)*0.2)
train_idx = indexes[:split_idx]
test_idx = indexes[split_idx:]


# 定义batch 方法：

def get_random_cached_bottlenecks(sess, batch_size, category):
    
    if category == 'training':
        idx = train_idx
    else:
        idx = test_idx
    
    class_count = len(idx)
    bottlenecks = []
    ground_truths = []
    filenames = []
    if batch_size >= 0:
        random_idx = np.random.permutation(len(idx))
        random_idx = random_idx[:batch_size]
        for i in random_idx:
            bottleneck = get_or_create_bottleneck(sess, i)
            bottlenecks.append(bottleneck)
            ground_truths.append(all_labels[i])
            filenames.append(all_files[i])
    else:
        # Retrieve all bottlenecks.
        for i in idx:
            bottleneck = get_or_create_bottleneck(sess, i)
            bottlenecks.append(bottleneck)
            ground_truths.append(all_labels[i])
            filenames.append(all_files[i])
                
    return bottlenecks, ground_truths, filenames


In [20]:
# sess = tf.InteractiveSession(graph=graph)

# summaries_dir = '/tmp/retrain_logs2/'

# with tf.Session(graph=graph) as sess:
    


In [21]:
summaries_dir = '/tmp/retrain_logs/'

In [22]:
# training_steps= 4000
dropout_keep_prob = 1

with tf.Session(graph=graph) as sess:
    # cache bottlenecks
#     cache_bottlenecks(sess)

    # summary
    merged = tf.summary.merge_all()

    train_writer = tf.summary.FileWriter(summaries_dir + '/train', sess.graph)
    validation_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/validation')
    train_saver = tf.train.Saver()

    # 
    # Set up all our weights to their initial default values.
    init = tf.global_variables_initializer()
    sess.run(init)
    # Run the training for as many cycles as requested on the command line.
    for i in range(FLAGS.how_many_training_steps):
        (train_bottlenecks, train_ground_truth, _) = get_random_cached_bottlenecks(
                sess, FLAGS.train_batch_size, 'training')
        # Feed the bottlenecks and ground truth into the graph, and run a training
        # step. Capture training summaries for TensorBoard with the `merged` op.
        train_summary, _ = sess.run(
            [merged, train_step],
            feed_dict={
                bottleneck_input: train_bottlenecks,
                ground_truth_input: train_ground_truth,
                keep_prob: dropout_keep_prob,
            })
        train_writer.add_summary(train_summary, i)

        # Every so often, print out how well the graph is training.
        is_last_step = (i + 1 == FLAGS.how_many_training_steps)
        if (i % FLAGS.eval_step_interval) == 0 or is_last_step:
            train_accuracy, cross_entropy_value = sess.run(
                [evaluation_step, cross_entropy_mean],
                feed_dict={
                    bottleneck_input: train_bottlenecks,
                    ground_truth_input: train_ground_truth
                })
            tf.logging.info('%s: Step %d: Cross entropy = %f, Train accuracy = %.1f%%' % (
                datetime.now(), i, cross_entropy_value, train_accuracy * 100))
            
            validation_bottlenecks, validation_ground_truth, _ = (
                get_random_cached_bottlenecks(sess, -1, 'validation'))

            # Run a validation step and capture training summaries for TensorBoard
            # with the `merged` op.
            validation_summary, validation_accuracy = sess.run(
                [merged, evaluation_step],
                feed_dict={
                    bottleneck_input: validation_bottlenecks,
                    ground_truth_input: validation_ground_truth
                })
            validation_writer.add_summary(validation_summary, i)
            tf.logging.info('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
                            (datetime.now(), i, validation_accuracy * 100,
                             len(validation_bottlenecks)))
    # 这个用于保存session中的模型和参数信息
    #     train_saver.save(sess, CHECKPOINT_NAME)

    #     # We've completed all our training, so run a final test evaluation on
    #     # some new images we haven't used before.
    test_bottlenecks, test_ground_truth, test_filenames = (
        get_random_cached_bottlenecks(sess, FLAGS.test_batch_size, 'testing'))
        
    test_accuracy, predictions = sess.run(
        [evaluation_step, prediction],
        feed_dict={
            bottleneck_input: test_bottlenecks,
            ground_truth_input: test_ground_truth
        })
    tf.logging.info('Final test accuracy = %.1f%% (N=%d)' %
                    (test_accuracy * 100, len(test_bottlenecks)))

INFO:tensorflow:2018-03-21 23:54:54.763299: Step 0: Cross entropy = 4.555732, Train accuracy = 15.0%
INFO:tensorflow:2018-03-21 23:54:55.524657: Step 0: Validation accuracy = 4.3% (N=1637)
INFO:tensorflow:2018-03-21 23:55:00.006247: Step 100: Cross entropy = 3.517713, Train accuracy = 28.0%
INFO:tensorflow:2018-03-21 23:55:00.655308: Step 100: Validation accuracy = 18.9% (N=1637)
INFO:tensorflow:2018-03-21 23:55:05.164823: Step 200: Cross entropy = 3.123281, Train accuracy = 35.0%
INFO:tensorflow:2018-03-21 23:55:05.826801: Step 200: Validation accuracy = 27.5% (N=1637)
INFO:tensorflow:2018-03-21 23:55:10.356965: Step 300: Cross entropy = 2.659822, Train accuracy = 47.0%
INFO:tensorflow:2018-03-21 23:55:11.007897: Step 300: Validation accuracy = 32.9% (N=1637)
INFO:tensorflow:2018-03-21 23:55:15.532596: Step 400: Cross entropy = 2.298745, Train accuracy = 63.0%
INFO:tensorflow:2018-03-21 23:55:16.192353: Step 400: Validation accuracy = 40.8% (N=1637)
INFO:tensorflow:2018-03-21 23:55:20

INFO:tensorflow:2018-03-21 23:58:36.687463: Step 4200: Cross entropy = 0.683311, Train accuracy = 93.0%
INFO:tensorflow:2018-03-21 23:58:37.360680: Step 4200: Validation accuracy = 73.7% (N=1637)
INFO:tensorflow:2018-03-21 23:58:42.052227: Step 4300: Cross entropy = 0.682368, Train accuracy = 93.0%
INFO:tensorflow:2018-03-21 23:58:42.709207: Step 4300: Validation accuracy = 74.0% (N=1637)
INFO:tensorflow:2018-03-21 23:58:47.387832: Step 4400: Cross entropy = 0.702028, Train accuracy = 93.0%
INFO:tensorflow:2018-03-21 23:58:48.051257: Step 4400: Validation accuracy = 74.0% (N=1637)
INFO:tensorflow:2018-03-21 23:58:52.746657: Step 4500: Cross entropy = 0.638100, Train accuracy = 96.0%
INFO:tensorflow:2018-03-21 23:58:53.416084: Step 4500: Validation accuracy = 74.2% (N=1637)
INFO:tensorflow:2018-03-21 23:58:58.089246: Step 4600: Cross entropy = 0.625572, Train accuracy = 93.0%
INFO:tensorflow:2018-03-21 23:58:58.752692: Step 4600: Validation accuracy = 74.4% (N=1637)
INFO:tensorflow:2018


```
INFO:tensorflow:2018-03-21 23:27:21.368419: Step 3300: Cross entropy = 0.091777, Train accuracy = 98.8%
INFO:tensorflow:2018-03-21 23:27:22.017629: Step 3300: Validation accuracy = 80.0% (N=1637)
INFO:tensorflow:2018-03-21 23:27:32.701135: Step 3400: Cross entropy = 0.097691, Train accuracy = 99.6%
INFO:tensorflow:2018-03-21 23:27:33.474558: Step 3400: Validation accuracy = 80.1% (N=1637)
INFO:tensorflow:2018-03-21 23:27:44.154611: Step 3500: Cross entropy = 0.085186, Train accuracy = 99.6%
INFO:tensorflow:2018-03-21 23:27:44.953435: Step 3500: Validation accuracy = 80.1% (N=1637)
```
batch size 256，  有20%的差距。。泛化能力还是太弱了。

```
INFO:tensorflow:2018-03-21 23:33:42.235579: Step 4800: Validation accuracy = 79.0% (N=1637)
INFO:tensorflow:2018-03-21 23:33:46.869099: Step 4900: Cross entropy = 0.099666, Train accuracy = 99.0%
INFO:tensorflow:2018-03-21 23:33:47.509601: Step 4900: Validation accuracy = 79.0% (N=1637)
INFO:tensorflow:2018-03-21 23:33:52.088484: Step 4999: Cross entropy = 0.087469, Train accuracy = 99.0%
INFO:tensorflow:2018-03-21 23:33:52.800459: Step 4999: Validation accuracy = 78.7% (N=1637)
INFO:tensorflow:Final test accuracy = 78.7% (N=1637)
```
 batch size 100，

## 5. error analysis

就是好奇，什么样的图片分类会错误。



In [23]:

error_files = []
label_names = list(image_lists.keys())

for i, pred in enumerate(predictions):
    label = test_ground_truth[i]
    if pred != label:
        filename = test_filenames[i]
        image_path = os.path.join(image_dir, label_names[label], filename)
        error_files.append((image_path, label_names[label], label_names[pred]))

NameError: name 'image_lists' is not defined

In [ ]:
1 - len(error_files) / len(test_filenames)

In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

# inspect preditions
cols = 4
rows = 7
fig = plt.figure(figsize=(4 * cols - 1, 6 * rows - 1))

for i, (filename, label, pred) in enumerate(error_files):
    c = i % cols
    r = i // cols
    ax = fig.add_subplot(rows, cols, i+1)
    ax.grid(False)
    ax.axis('off')
    image = Image.open(filename)
    # can use crop here
    image = image.resize((100, 100), Image.ANTIALIAS)
    ax.imshow(np.asarray(image))
    ax.set_title("true: {}\n pred: {}".format(label, pred))
plt.show()

# roses：玫瑰
# tulips：郁金香
# sunflowers： 向日葵
# dandelion： 蒲公英
# daisy： 雏菊
